# Circular Random Projection
- This is a software experiment to check out how to do circular random projection
- The idea is that instead of randomizing a huge matrix, we can make a smaller seed matrix and populate the larger matrix with smaller circular permutations of the larger matrix

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import comb
import itertools

# Import library
import os
import sys

# Gets directory where script was launched from
script_dir = os.getcwd()  
script_dir = script_dir + "/../lib/"
print(f"VSA Library: {script_dir}")

# Add the directory to Python's search path
sys.path.append(script_dir)  

import vsa

VSA Library: /home/rantonio/chronomatica/math/../lib/


# Useful Functions

In [2]:
# Random indexing function to make an NxM matrix
def random_indexing(N, M):
    # Create aranged indices
    matrix = np.arange(N * M)
    # Shuffle the matrix
    np.random.shuffle(matrix)
    matrix = matrix.reshape(N, M)
    # Create the bipolar matrix
    matrix[matrix < (N * M) // 2] = -1
    matrix[matrix >= (N * M) // 2] = 1
    return matrix

In [3]:
def numbered_indexing(N, M):
    # Create an NxM matrix with numbers from 0 to N*M-1
    matrix = np.arange(N * M).reshape(N, M)
    return matrix

In [4]:
def circular_block(seed_matrix):
    return np.roll(seed_matrix.flatten(), shift=1).reshape(seed_matrix.shape)

In [5]:
def matrix_expansion(sub_matrix, X, Y):
    # Copy sub matrix
    base_matrix = sub_matrix

    # Stitching the bigger matrix
    blocks = []

    # Horizontal expansion
    for i in range(Y):
        row_blocks = []
        # Vertical expansion
        for j in range(X):
            base_matrix = circular_block(base_matrix)
            row_blocks.append(base_matrix)
        
        # Stack vertically
        blocks.append(np.vstack(row_blocks))

    # Stack horizontally
    final_matrix = np.hstack(blocks)

    return final_matrix

# Creating a Simple Test Case
- Let's try for now a simple input vector of $1 \times 16$ multiplied to a matrix $16 \times 16$ elements.
- However the matrix $16 \times 16$ is created using sub $4 \times 4$ matrices that were pre-generated with 1s and -1s
- There is a random indexing function to ensure the number of 1s and -1s are equal

In [6]:
# Create base vector
vector = numbered_indexing(1, 16)
print(f"Base vector: {vector}")

# Create sub-matrix
sub_matrix = numbered_indexing(4, 4)
print(f"Sub-matrix:\n{sub_matrix}")

Base vector: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
Sub-matrix:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


- The example below shows how the indices change per sub-matrix that is permuted

In [7]:
print(f"Sub-matrix iter 0:\n{sub_matrix}")

permute_sub_matrix = circular_block(sub_matrix)
print(f"Sub-matrix iter 1:\n{permute_sub_matrix}")

permute_sub_matrix = circular_block(permute_sub_matrix)
print(f"Sub-matrix iter 2:\n{permute_sub_matrix}")

permute_sub_matrix = circular_block(permute_sub_matrix)
print(f"Sub-matrix iter 3:\n{permute_sub_matrix}")

Sub-matrix iter 0:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
Sub-matrix iter 1:
[[15  0  1  2]
 [ 3  4  5  6]
 [ 7  8  9 10]
 [11 12 13 14]]
Sub-matrix iter 2:
[[14 15  0  1]
 [ 2  3  4  5]
 [ 6  7  8  9]
 [10 11 12 13]]
Sub-matrix iter 3:
[[13 14 15  0]
 [ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


- The matrix below shows the indices of how those permutations are stitched together.
- So the sub-matrices from the larger block needs to be permutated vertically first before moving horizontally.
- The pattern of the indices should be sufficient to show how the permutations work.

In [8]:
expanded_matrix = matrix_expansion(sub_matrix, 4, 4)
expanded_matrix

array([[15,  0,  1,  2, 11, 12, 13, 14,  7,  8,  9, 10,  3,  4,  5,  6],
       [ 3,  4,  5,  6, 15,  0,  1,  2, 11, 12, 13, 14,  7,  8,  9, 10],
       [ 7,  8,  9, 10,  3,  4,  5,  6, 15,  0,  1,  2, 11, 12, 13, 14],
       [11, 12, 13, 14,  7,  8,  9, 10,  3,  4,  5,  6, 15,  0,  1,  2],
       [14, 15,  0,  1, 10, 11, 12, 13,  6,  7,  8,  9,  2,  3,  4,  5],
       [ 2,  3,  4,  5, 14, 15,  0,  1, 10, 11, 12, 13,  6,  7,  8,  9],
       [ 6,  7,  8,  9,  2,  3,  4,  5, 14, 15,  0,  1, 10, 11, 12, 13],
       [10, 11, 12, 13,  6,  7,  8,  9,  2,  3,  4,  5, 14, 15,  0,  1],
       [13, 14, 15,  0,  9, 10, 11, 12,  5,  6,  7,  8,  1,  2,  3,  4],
       [ 1,  2,  3,  4, 13, 14, 15,  0,  9, 10, 11, 12,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  1,  2,  3,  4, 13, 14, 15,  0,  9, 10, 11, 12],
       [ 9, 10, 11, 12,  5,  6,  7,  8,  1,  2,  3,  4, 13, 14, 15,  0],
       [12, 13, 14, 15,  8,  9, 10, 11,  4,  5,  6,  7,  0,  1,  2,  3],
       [ 0,  1,  2,  3, 12, 13, 14, 15,  8,  9, 10,

# Ideal Matrix Multiplication
- In the ideal matrix multiplication, it's simply the vector multiplied by the matrix

In [9]:
ideal_matmul = np.matmul(vector, expanded_matrix)
ideal_matmul

array([[804, 924, 980, 972, 772, 876, 916, 892, 804, 892, 916, 876, 900,
        972, 980, 924]])

# Tiled Circular Matrix Projection
- So the idea of the tiled circular matrix projection is that we don't need to store the entire permuted matrices but start only from a seed matrix, multiply things one by one.
- In this case suppose we have an input vector of size $1 \times 16$
- Then we have a sub-matrices of size $4 \times 4$ and the thing is we want to slide and circular permute this unto the input vector as if we did the multiplication of the vector and the expanded $16 \times 16$ matrix.
- Since we have $4 \times 4$ sub-matrices, then it's sufficient to say to cut the vector into four $1 \times 4$ sub-vectors.
- Then we multiply the sub-vectors to the column-wise sub-matrices first as it produces $1 \times 4$ sub-vectors for the output and we eventually concatenate them to make the desired $1 \times 16$ output vector.

## First, cut input vector into sub-vectors
- Technically easier done to just reshape the vector into $4 \times 4$ where each row is one sub-vector.

In [10]:
print(f"Orig vector:\n{vector}")

split_vector = vector.reshape(4, 4)
print(f"Split vector:\n{split_vector}")

Orig vector:
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
Split vector:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


# Iterative Circular Projections
- Since we only, supposedly, store the seed sub-matrix, then we need to iterate through the permutations one by one.
- In this example we need to iterate 16 times.

In [12]:
# Initialize expected output
expected_output = np.zeros((4, 4), dtype=int)

# Iterate through the permuted sub-matrices
permuted_sub_matrix = sub_matrix.copy()

for i in range(4):
    for j in range(4):
        # Get the permuted sub-matrix
        permuted_sub_matrix = circular_block(permuted_sub_matrix)
        
        # Perform matrix multiplication only on the sub-matrix and sub-vector
        result = np.matmul(split_vector[j, :], permuted_sub_matrix)
        
        # Store the result in the expected output location
        # Technically it is an output sub-vector
        expected_output[i] = expected_output[i] + result

expected_output = expected_output.reshape(1, 16)
print(f"Expected output after permuted sub-matrices:\n{expected_output}")

# Compare the ideal matrix multiplication with the expected output
print(expected_output == ideal_matmul)

Expected output after permuted sub-matrices:
[[804 924 980 972 772 876 916 892 804 892 916 876 900 972 980 924]]
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]]


## Conclusion
- The circular random projection does multiple matrices instead but this time we only need to store a single seed sub matrix $4 \times 4$ instead of storing a larger $16 \times 16$ matrix.

# Checking for Matrix Viability
- Some random matrices that are bipolar $\{+1,-1\}^D$ may not be viable at times if the permutations, by chance become the same.
- For example, a small vector $[+1,-1,+1,-1,+1,-1,+1,-1]$ when permuted twice you get the same vector.
- Therefore, to make sure we get the proper random matrix projection, it is necessary that the matrix needs to have column vectors as linearly independent. We can check linear independence by checking the rank of the matrix.
- Take note that linear independence means that a set of vectors $\{v_1, v_2, ... v_n\} \in \mathbb{R}^{D}$ is linearly independent if and only if for a given set of scalars $\{a_1, a_2, ... a_n\}$:
$$a_1v_1 + a_2v_2 + ... + a_nv_n = 0 $$
- Has only one trivial solution when $a_1 = a_2 = ... = a_n = 0$.
- Therefore if there is by chance there is a vector dependency, then that's not good.


In [13]:
# Sample of a linearly independent matrix
# Observe that these are circularly permuted
# row-vectors but we will tranpose it
matA = np.array([
    [1,-1,1,-1,1,1,-1,-1],
    [-1,1,-1,1,-1,1,1,-1],
    [-1,-1,1,-1,1,-1,1,1],
])

matA = matA.T
rankmatA = np.linalg.matrix_rank(matA)

# The rank of the matrix needs to be
# the same size of the columns
rankmatA == matA.shape[1]

np.True_

In [14]:
# Sample of a not linearly independent matrix

matA = np.array([
    [1,-1,1,-1,1,-1,1,-1],
    [-1,1,-1,1,-1,1,-1,1],
    [1,-1,1,-1,1,-1,1,-1],
])

matA = matA.T
rankmatA = np.linalg.matrix_rank(matA)

# The rank of the matrix needs to be
# the same size of the columns
rankmatA == matA.shape[1]

np.False_

- So from the above examples, indeed if there are permutations that eventually lead to linearly dependent vectors are not viable, and it can happen if we are not careful.
- We can approach this in two ways: (1) we sample a vector and see if the maximum expanded matrix would still be linearly dependent, or (2) we list all possibilities and list the viable ones only.
- This might be highly dependent on whether vector is short or long. If it's short enough, then there's a good posibility to list all. For example, if the vector (or submatrix) is like 16 elements long, then 16 taken 8 permutations to 12870 combinations.

In [15]:
n_permutations = comb(16, 8)
n_permutations

12870

- This means there is probability some number out of the total 12,870 that may cause a combination that is limited.
- In the following experiment let's try to generate bigger matrices of 16 vectors that were randomly generated.
- Take note, the vectors have equal $+1$ and $-1$ (i.e., 8 times each).

In [16]:
def measure_correct_rank(N, vsize):
    # Iterate N times to track if the rank changes
    match_score = 0
    
    for n in range(N):
        # Initialize matrix
        matrix = np.zeros((vsize,vsize))
        
        # Generate vector seed
        vector_seed = vsa.gen_hv(vsize, type='bipolar')
        
        # Make permutations into vector seed
        for i in range(len(matrix)):
            matrix[i] = np.roll(vector_seed,i)
        
        # Transpose to make the vectors columns
        matrix = matrix.T
        
        # Check the rank
        rank_matrix = np.linalg.matrix_rank(matrix)
        if(rank_matrix == vsize):
            match_score += 1

    match_score_percent = (match_score / N)*100
    print(f"Percentage of matches: {match_score_percent:.2f}")
    return


In [17]:
measure_correct_rank(100, 16)

Percentage of matches: 62.00


- Clearly, this just shows that there are some combinations that give proper matrices or not
- Let's check out the of it happening if the vector size was larger

In [18]:
measure_correct_rank(100, 128)

Percentage of matches: 88.00


In [20]:
measure_correct_rank(100, 512)

Percentage of matches: 90.00


- So if the seed vectors or matrices are larger, there is a higher chance that we get more linearly independent components.
- The thing is, in our ResNet + HDC requirement, we need a $4,096 \times 512$ projection matrix which implies we need 512 $4,096 \times 1$ vectors that are linearly independent.
- To generate a $4,096 \times 1$ vector with a single $16 \times 1$ vector, we need $\frac{4,096}{16}= 256$ seed vectors of $16 \times 1$ to create one $4,096 \times 1$ vector.
- The problem is we can only permute $16$ times and if we need $512$ of those $4,096 \times 1$ vectors then for a single set of $256$ seed-vectors we can only make $4,096 \times 16$ out of the $16$ permutations.
- To encompass up to 512 $4,096 \times 1$ vectors, we need $\frac{512}{16} = 32$ more seed sets then in total we need $256 \times 32 = 8,192$ seed vectors.
- **This makes the $16\times 1$ vectors not applicable because we need to generate $8,192$ seed vectors but sadly we can only have $12,870$ combinations from the $16 \times 1$ vectors with equal $+1$ and $-1$ elements. Take note that not all of the $12,870$ combinations are also valid since some of them generate matrices with linear dependencies.** 

- The safest way to do so is to use 8 $512 \times 1$ seed vectors to generate one $4,096 \times 1$
- Because we can permute at the maximum $512$ times too, then we literally only need 8 seed vectors to cover all $4,096 \times 512$ matrix projection.
- Moreover, since there is a high probability of making a fully linearly independent matrix, then we definitely would have a gazillion choices of an appropriate set! 